In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import platform
import platform
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
##그래프의 결과를 출력 세션에 나타나게 하자
%matplotlib inline 
plt.rcParams['axes.unicode_minus'] = False
if platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows':
    path ='C:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Can't find your system.")

### 머신러닝을 통한 결측치(건축년도)제거

In [2]:
#읍면동 구분하는 함수
def sort_dong(df):
    a=df.split(" ")
    if a[1][-1]=='동':
        return '동'
    elif a[1][-1]=='면':
        return '면'
    else :
        return '읍'
#동 구분하는 함수
def sort_dong2(df):
    a=df.split(" ")
    return a[1]
#년도 추출
def year(df):
    return df[:-2]
#평수 분해
def pyeong(df):
    if df< 33:
        return '10평 미만'
    elif df>= 33 and df < 66:
        return '10~20평'
    elif df>= 66 and df < 99:
        return '20~30평'
    elif df>= 99 and df < 132:
        return '30~40평'
    elif df>= 132 and df < 165:
        return '40~50평'
    elif df>= 165 and df < 198:
        return '50~60평'
    else:
        return '60평 이상'
def mon(df):
    return df[-2:]
def pyeong2(df):
    return int(df/3.3)
def riri(df):
    return int(float(df.replace(',','')))
def rep(df):
    if df == '-':
        return 0
    elif df.find('m이상') != -1:
        return df.replace('m이상','')
    else:
        return df.replace('m미만','')

In [3]:
danta = pd.read_csv('5.세종시_단독다가구(매매)_실거래가.csv')

#### 건축년도 결측치 확인

In [4]:
danta[danta['건축년도'].isnull()]

,계약년월,계약일,시군구,주택유형,도로조건,연면적(㎡),대지면적(㎡),거래금액(만원),건축년도,해제사유발생일
37,201702,18,세종특별자치시 연서면 국촌리,단독,8m미만,104.60,304.0,"7,000",NaN,NaN
44,201702,26,세종특별자치시 조치원읍 신흥리,단독,8m미만,41.31,152.0,"2,000",NaN,NaN
68,201703,15,세종특별자치시 전동면 심중리,단독,8m미만,61.16,347.0,"12,353",NaN,NaN
278,201711,20,세종특별자치시 장군면 금암리,단독,8m미만,88.45,567.0,"27,400",NaN,NaN
827,202003,18,세종특별자치시 장군면 금암리,단독,8m미만,185.92,1104.0,"90,000",NaN,20200519.0
1021,202009,24,세종특별자치시 연기면 보통리,다가구,8m미만,493.86,505.0,"90,000",NaN,20201015.0
1023,202009,24,세종특별자치시 연기면 보통리,다가구,8m미만,493.86,532.0,"90,000",NaN,20201015.0
1025,202009,24,세종특별자치시 연기면 보통리,다가구,8m미만,493.86,595.2,"90,000",NaN,20201015.0
1026,202009,24,세종특별자치시 연기면 보통리,다가구,8m미만,493.86,619.8,"90,000",NaN,20201015.0


#### 결측치 제거를 위한 데이터 정제

In [5]:
danta["평"]=danta["연면적(㎡)"].apply(pyeong)
danta['읍면동구분']=danta['시군구'].apply(sort_dong)
danta['읍면동']=danta['시군구'].apply(sort_dong2)
danta["도로"]=danta["도로조건"].apply(rep)

#### 머신러닝을 처리하기 위한 문자의 숫자화

In [7]:
from sklearn.preprocessing import LabelEncoder
labelencoder_x= LabelEncoder()
y = danta["시군구"]
danta["시군구2"]=labelencoder_x.fit_transform(y)
z = danta["주택유형"]
danta["주택유형2"]=labelencoder_x.fit_transform(z)
danta['거래금액2(만원)']= danta['거래금액(만원)'].apply(riri)

#### 선형회귀를 통한 머신러닝

In [8]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

In [9]:
danta_null=danta[danta['건축년도'].isnull()&danta['해제사유발생일'].isnull()]
danta_null

,계약년월,계약일,시군구,주택유형,도로조건,연면적(㎡),대지면적(㎡),거래금액(만원),건축년도,해제사유발생일,평,읍면동구분,읍면동,도로,시군구2,주택유형2,거래금액2(만원)
37,201702,18,세종특별자치시 연서면 국촌리,단독,8m미만,104.60,304.0,"7,000",NaN,NaN,30~40평,면,연서면,8,53,1,7000
44,201702,26,세종특별자치시 조치원읍 신흥리,단독,8m미만,41.31,152.0,"2,000",NaN,NaN,10~20평,읍,조치원읍,8,108,1,2000
68,201703,15,세종특별자치시 전동면 심중리,단독,8m미만,61.16,347.0,"12,353",NaN,NaN,10~20평,면,전동면,8,85,1,12353
278,201711,20,세종특별자치시 장군면 금암리,단독,8m미만,88.45,567.0,"27,400",NaN,NaN,20~30평,면,장군면,8,65,1,27400


In [10]:
#null 값이 아닌 것만 뽑아오기
idx_not=danta[danta['건축년도'].isnull()].index
danta_notnull=danta.drop(idx_not)

In [11]:
X = danta_notnull[["계약년월","연면적(㎡)","대지면적(㎡)","거래금액2(만원)","도로","시군구2","주택유형2"]]
y = danta_notnull['건축년도']

In [12]:
lm.fit(X,y)

LinearRegression()

In [13]:
X2 = danta_null[["계약년월","연면적(㎡)","대지면적(㎡)","거래금액2(만원)","도로","시군구2","주택유형2"]]

In [14]:
predictions = lm.predict(X2)
predictions=predictions.astype(int)

In [15]:
for i in range(len(danta_null.index)):
    danta.loc[ danta_null.index[i] , "건축년도" ]=int(predictions[i])

#### 건축년도 예측

In [16]:
danta.loc[[37,44,68,278]]

,계약년월,계약일,시군구,주택유형,도로조건,연면적(㎡),대지면적(㎡),거래금액(만원),건축년도,해제사유발생일,평,읍면동구분,읍면동,도로,시군구2,주택유형2,거래금액2(만원)
37,201702,18,세종특별자치시 연서면 국촌리,단독,8m미만,104.60,304.0,"7,000",1985.0,NaN,30~40평,면,연서면,8,53,1,7000
44,201702,26,세종특별자치시 조치원읍 신흥리,단독,8m미만,41.31,152.0,"2,000",1978.0,NaN,10~20평,읍,조치원읍,8,108,1,2000
68,201703,15,세종특별자치시 전동면 심중리,단독,8m미만,61.16,347.0,"12,353",1982.0,NaN,10~20평,면,전동면,8,85,1,12353
278,201711,20,세종특별자치시 장군면 금암리,단독,8m미만,88.45,567.0,"27,400",1986.0,NaN,20~30평,면,장군면,8,65,1,27400
